# InfluxDB Testing

In [3]:
from influxdb_client import InfluxDBClient
import dotenv
import os

In [32]:
dotenv.load_dotenv('.env')
url = "http://10.10.120.16:8086"
token = os.getenv("INFLUXDB_TOKEN")
org = os.getenv("INFLUXDB_ORG")

In [79]:
query = 'from(bucket:"homeassistant") |> range(start: -11m, stop: -1m) \
  |> filter(fn: (r) => r["entity_id"] == "sonoff_1001e01c1e_power")\
  |> filter(fn: (r) => r["_field"] == "value")\
  '

client = InfluxDBClient(url="http://10.10.120.16:8086", token=token, org=org)
query_api = client.query_api()
tables = query_api.query(query=query)

In [ ]:
measurement = []
i = 0
for table in tables:
    for row in table.records:
        print(row.values["_time"]) # row.values is a dictionary
        print(row.values["_value"])
        value = row.values["_value"]
        measurement.append(value)

In [ ]:
measurement

In [82]:
average = sum(measurement) / len(measurement)

In [83]:
average

23.792347826086953

Generating Random Data for Analysis

In [111]:
import random

random_int_c7 = [random.randint(2200,3000)/100 for i in range(200)]
random_int_c3 = [random.randint(2250,3000)/100 for i in range(200)]
random_int_c0 = [random.randint(2300,3000)/100 for i in range(200)]

In [112]:
def write_random(c, rand_list, writer):
    hrs = 9
    for i in rand_list:
        hrs += 1
        writer.write(f'Timestamp, {hrs}, {i}, {c}\n')


with open('influx-test.csv','w') as csvfile:
    csvfile.write('Timestamp, Hrs, Value, C-State\n')
    write_random('C7', random_int_c7, csvfile)
    write_random('C3', random_int_c3, csvfile)
    write_random('C0', random_int_c0, csvfile)
